# Simple Data Source Q&A

This is a sample implementation of an analytical Agent for Tableau built with Langgraph

### Dependencies and Environment Variables

First we begin with the dependencies and environment variables needed to run the Agent graph

In [6]:
import sys
# Print current working directory
print("Current Working Directory:", os.getcwd())

# Print the Python executable path
print("Python Executable Path:", sys.executable)


Current Working Directory: /Users/josephflu/ztemp/t
Python Executable Path: /Users/josephflu/ztemp/t/env_prod/bin/python


In [8]:
import os
from dotenv import load_dotenv
from IPython.display import display, Markdown

# base langchain library imports
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

# langchain_tableau imports
from langchain_tableau.tools.simple_datasource_qa import initialize_simple_datasource_qa

# environment variables available to current process and sub processes
load_dotenv()
# variables for authenticating and interacting with a Tableau site
tableau_domain = os.environ['TABLEAU_DOMAIN']
tableau_site = os.environ['TABLEAU_SITE']
tableau_jwt_client_id = os.environ['TABLEAU_JWT_CLIENT_ID']
tableau_jwt_secret_id = os.environ['TABLEAU_JWT_SECRET_ID']
tableau_jwt_secret = os.environ['TABLEAU_JWT_SECRET']
tableau_api_version = os.environ['TABLEAU_API_VERSION']
tableau_user = os.environ['TABLEAU_USER']
# the target data source for this Tool
datasource_luid = os.environ['DATASOURCE_LUID']
# variables to control LLM models for the Agent and Tools
open_api_key = os.environ["OPENAI_API_KEY"]
agent_llm_model = os.environ['AGENT_MODEL']
tooling_llm_model = os.environ['TOOLING_MODEL']

### Agent Graph

Then we define the Agent graph using Langgraph's prebuilt `create_react_agent` combined with the a tool to query a Tableau data source for conversational analytics

In [9]:
# configure running model for the agent
llm = ChatOpenAI(
    model=agent_llm_model,
    api_key=open_api_key,
    temperature=0,
    verbose=True,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

# Tableau VizQL Data Service Query Tool
analyze_datasource = initialize_simple_datasource_qa(
    domain=tableau_domain,
    site=tableau_site,
    jwt_client_id=tableau_jwt_client_id,
    jwt_secret_id=tableau_jwt_secret_id,
    jwt_secret=tableau_jwt_secret,
    tableau_api_version=tableau_api_version,
    tableau_user=tableau_user,
    datasource_luid=datasource_luid,
    tooling_llm_model=tooling_llm_model
)

# load the List of Tools to be used by the Agent
tools = [ analyze_datasource ]

# set agent debugging state
if os.getenv('DEBUG') == '1':
    debugging = True
else:
    debugging = False

# define the agent graph
query_agent = create_react_agent(
    model=llm,
    tools=tools,
    debug=debugging
)

### Interacting with the Analytical Agent

After constructing an Analytical Agent with the power to query a Tableau data source, we proceed to ask questions, set tasks and issue commands to the Agent

In [10]:
# question or task sent to the agent
message_string = 'show me average discount, total sales and profits by region sorted by profit'

# Run the agent
messages = query_agent.invoke({"messages": [("human", message_string)]})
agent_message = "\n\nAnalytics Agent:\n\n" + messages['messages'][3].content
display(Markdown(agent_message))

[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('human',
               'show me average discount, total sales and profits by region '
               'sorted by profit')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('human',
  'show me average discount, total sales and profits by region sorted by '
  'profit')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='show me average discount, total sales and profits by region sorted by profit', additional_kwargs={}, response_metadata={}, id='e6bf3993-46f0-466f-ade9-c9ca3d58d066')]}
[1:tasks] Starting 1 task for step 1:
- agent -> {'is_last_step': False,
 'messages': [HumanMessage(content='show me average discount, total sales and profits by region sorted by profit', additional_kwargs={}, response_metadata={}, id='e6bf3993-46f0-466f-ade9-c9ca3d58d066')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with 



Analytics Agent:

Here are the average discount, total sales, and profits by region, sorted by profit:

| Region   | Average Discount | Total Sales  | Total Profit  |
|----------|------------------|--------------|---------------|
| West     | 0.11             | 739,813.61   | 110,798.82    |
| East     | 0.14             | 691,828.17   | 94,883.26     |
| South    | 0.15             | 391,721.91   | 46,749.43     |
| Central  | 0.24             | 503,170.67   | 39,865.31     |

This data is derived from the sales data source, providing insights into the performance of different regions based on sales and profit metrics.